In [1]:
import csv
import json
import sys
import pandas as pd
import os
import copy
import math
import json

from collections import defaultdict
import numpy as np
import itertools as it


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz    
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip3 install -q findspark
!pip3 install -q pyspark
!pip3 install -q kaggle
!pip3 install bitmap
from bitmap import BitMap

from google.colab import files
print("Upload kaggle.json to use kaggle APIs")
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

# !curl -OL https://raw.githubusercontent.com/Bertonc98/ProgettoAMD/main/sample.csv

!kaggle datasets files "ashirwadsangwan/imdb-dataset"
!kaggle datasets download "ashirwadsangwan/imdb-dataset" -f "title.principals.tsv.gz"
!unzip "title.principals.tsv.gz.zip"
!rm "title.principals.tsv.gz.zip"
!gunzip -k "title.principals.tsv.gz"
!rm "title.principals.tsv.gz"

     |████████████████████████████████| 212.3MB 64kB/s 
     |████████████████████████████████| 204kB 18.9MB/s 
Upload kaggle.json to use kaggle APIs


Saving kaggle.json to kaggle.json
kaggle.json
name                      size  creationDate         
-----------------------  -----  -------------------  
title.ratings.tsv.gz       5MB  2019-11-18 18:36:47  
name.basics.tsv.gz       180MB  2019-11-18 18:36:47  
title.basics.tsv.gz      108MB  2019-11-18 18:36:47  
title.principals.tsv.gz  290MB  2019-11-18 18:36:47  
title.akas.tsv.gz        158MB  2019-11-18 18:36:47  
 98% 282M/287M [00:06<00:00, 53.2MB/s]
100% 287M/287M [00:06<00:00, 49.0MB/s]
Archive:  title.principals.tsv.gz.zip
  inflating: title.principals.tsv.gz  


In [3]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

home = os.environ["SPARK_HOME"]
os.environ["PYTHONPATH"] = home+"/python"+home+"/python/lib/py4j-0.10.9-src.zip:"+os.environ["PYTHONPATH"]

In [4]:
import findspark
from pyspark.sql import SparkSession
import pyspark
from pyspark import SparkConf
findspark.init()
conf = SparkConf()
conf.set("spark.driver.memory", "5g")
conf.set("spark.executor.memory", "5g")
spark = SparkSession.builder.config(conf = conf).master("local[*]").getOrCreate()

In [5]:
sourceFile = 'title.principals.tsv'

data = spark.read.options(delimiter = "\t", header=True).csv(sourceFile)

from pyspark.sql.window import Window

from pyspark.sql.functions import monotonically_increasing_id,row_number

rdd_result = data.select(['nconst', 'category'])\
            .where("category == 'actress' or category == 'actor' ")\
            .select(['nconst'])\
            .distinct()

rdd_result = rdd_result.withColumn("idx",row_number().over(Window.orderBy(monotonically_increasing_id())))

In [53]:
# this dictionary maps all the items to an int
itemToInt = {key:value for (key, value) in rdd_result.collect()}

In [7]:
filter_actors = lambda row: row["category"] in ["actor", "actress"]

map_actors = lambda row: (row["tconst"], itemToInt[row["nconst"]]) 

rdd_basket = data.rdd.filter(filter_actors)\
                 .map(map_actors)\
                 .groupByKey()\
                 .mapValues(list)\
                 .values()
# res = rdd_basketId.collect()
# with open('preprocessed.csv', 'w', newline='') as f:
#     writer = csv.writer(f, delimiter=";")
#     writer.writerows(res)

In [63]:
with open('intToItem.csv', 'w', encoding = "utf-8") as f:
    writer = csv.writer(f, delimiter=";")
    writer.writerows(zip(itemToInt.values(), itemToInt.keys()))
del itemToInt

In [10]:
sample = rdd_basket.sample(False, 0.001)
sample.cache()

PythonRDD[31] at RDD at PythonRDD.scala:53

In [11]:
sample.count()

3691

In [12]:
# SON

# phase 1 scan all baskets 
# -input: id_movie, list(int_actor)
# -compute frequency (count)
# -count combinations (size 2) into hashmap
# -filter frequent singletons (count >= s)
# -output: int_actor: count

def hashF(element, size):
    return hash(element)%size

def get_combinations(basket, size):
    return it.combinations(sorted(basket), size)

def update_frequency(l, item):
    if item + 1 > len(l):
        l.extend((item+1-len(l))*[0])
    l[item] += 1
    
def singleton_pairs_frequency(chunk, hash_size, threshold):
    singleton_freq = []
    hashmap = np.zeros(hash_size)
    for basket in chunk:
        basket = list(map(int, basket))
        for item in basket:
            update_frequency(singleton_freq, item)
        for pair in get_combinations(basket, 2):
            hashmap[hashF(pair, hash_size)] += 1
    return filter_singletons(singleton_freq, threshold), hashmap

# this function filter the given singletons with the absolute freqeuncy, deleting the ones with
# frequency below the threshold
def filter_singletons(singleton, threshold):
    return [(iid,) for iid, freq in enumerate(singleton) if freq>threshold]    

# this function collapse the hashmap given in input into a bitmap of size "size" and set a 1 in the bitmap if the value in
# the hashmap is over the the given threshold
def hash_to_bitmap(hashmap, size, threshold):
    bmap = BitMap(size)
    for position in range(len(hashmap)):
        if hashmap[position] >= threshold:
            bmap.set(position)
    del hashmap
    return bmap

def get_joins(tuples, k):
    #implement join in this way
    #to obtain an element of lenght k:
    #join 2 elements of length k-1 with the first k-1 elements in common
    l = []
    for i in range(len(tuples)-1):
        for j in range(i+1, len(tuples)):
            if(tuples[i][:k-2] == tuples[j][:k-2]):
                l += [tuples[i] + tuples[j][k-2:]]
    return l

def get_kth(chunk, precedents, hash_size, bmap, threshold, k):
#     tuple of length k with corresponding bitmap value = 1
    kth = [tpl for tpl in get_joins(precedents, k) if bmap[hashF(tpl, hash_size)]]
    
#     counts the absolute frequency of the tuple in kth
    counters = np.zeros(len(kth))
#     hashmap for the tuples of length k+1
    next_hashmap = np.zeros(hash_size)
    
    for basket in chunk:
        basket = list(map(int, basket))
        tpls = list(get_combinations(basket, k))
        for i in range(len(kth)):
            if kth[i] in tpls:
                counters[i] += 1
        for kth_tuples in get_combinations(basket, k+1):
            next_hashmap[hashF(kth_tuples, hash_size)] += 1

    return [kth[i] for i in range(len(kth)) if counters[i]>=threshold], next_hashmap


In [13]:
def pcy(threshold, hash_size):
# def main(chunk, threshold, hash_size):
  def main(partition):
      generator, chunk = it.tee(partition, 2)
      frequent_sets = []

  #     phase 1: compute frequency of singletons and pairs; filter nonfrequent singletons
      singleton, hashmap = singleton_pairs_frequency(chunk, hash_size, threshold)
  #     phase 2: collpse the hahsmap into a bitmap
      bitmap = hash_to_bitmap(hashmap, hash_size, threshold)
  #     phase 3
  #     frequent_sets contains all the sets of all the kth length that are frequent
  #     bitmap is the bitmap mapping the sets of length k+1
  #     frequent_sets, bitmap = frequent_couple(data, singleton, hash_size, bitmap, threshold)
      frequent_sets += singleton
      
      k = 2
      result = ["placeholder"]
      while result != []:
          precedents = [i for i in frequent_sets if len(i) == k-1 ]
          chunk = copy.copy(generator)
          result, hashmap = get_kth(chunk, precedents, hash_size, bitmap, threshold, k)
          bitmap = hash_to_bitmap(hashmap, hash_size, threshold)
          frequent_sets = frequent_sets + result
          print("Found frequent itemset of length: ",k)
          k += 1
          
      return frequent_sets
  return main

In [18]:
p = 1/sample.getNumPartitions()
#to fix
s = math.ceil(sample.count()/1000)
# threshold = math.ceil(0.95 * s * p)
threshold = 1
hash_size = 504
candidates = sample.mapPartitions(pcy(threshold, hash_size))\
                   .map(lambda item: (1,[item]))\
                   .reduceByKey(lambda a, b: list(set(a + b )))\
                   .mapValues(list)\
                   .values()\
                   .collect()[0]

In [71]:
def son(candidates):
  def son_main(partition):
    count = {item: 0 for item in candidates}
    for basket in partition:
      for c in candidates:
          if c in get_combinations(basket, len(c)):
            count[c] += 1
    return [(k, v) for k, v in count.items()]
  return son_main

frequent = sample.mapPartitions(son(candidates))\
                 .reduceByKey(lambda a,b: a+b)\
                 .filter(lambda item: item[1] >= s)\

In [ ]:
f = csv.reader(open("intToItem.csv", encoding="utf-8", newline="\n"), delimiter=";")
intToItem = { int(k):v for (k, v) in f}

In [77]:
def getItemIdFromInt(l):
  return [intToItem[k] for k in l[0]]

frequent.map(getItemIdFromInt).collect()

[['nm4489288'],
 ['nm1294558'],
 ['nm0959926'],
 ['nm4434202'],
 ['nm0576967'],
 ['nm0136994', 'nm0717265'],
 ['nm0164818'],
 ['nm0115538'],
 ['nm10608973'],
 ['nm5438441', 'nm4434202'],
 ['nm0555829'],
 ['nm5438441', 'nm4434202', 'nm10608973'],
 ['nm7472745'],
 ['nm4755526'],
 ['nm0717265', 'nm0164818'],
 ['nm0239114'],
 ['nm5438441'],
 ['nm0071386'],
 ['nm0127925'],
 ['nm1131544'],
 ['nm4755526', 'nm4489288'],
 ['nm4396059'],
 ['nm5438441', 'nm10608973'],
 ['nm0276910'],
 ['nm4434202', 'nm10608973'],
 ['nm0136994', 'nm0164818'],
 ['nm0276910', 'nm0786939'],
 ['nm0152839'],
 ['nm0298439'],
 ['nm1588274'],
 ['nm0524456'],
 ['nm5014030'],
 ['nm0737226'],
 ['nm0192504'],
 ['nm0786939', 'nm0576967'],
 ['nm0239134'],
 ['nm0717265'],
 ['nm0039916'],
 ['nm0136994', 'nm0717265', 'nm0164818'],
 ['nm0136994'],
 ['nm0026164'],
 ['nm0786939']]